In [5]:
!pip install dash
!pip install dash-leaflet

In [6]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_leaflet as dl
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import mapping
import leafmap as leafmap
from shapely.geometry import shape
from dash_extensions.javascript import assign
import ipywidgets as widgets

In [9]:

# Utility Function to Read API Responses

def read_response(t):
    try:
        data = t.json() #This will convert the response to a json object
        return data
    except requests.exceptions.JSONDecodeError:
        print("Risposta non valida JSON!")
        print("Contenuto della risposta:", t.text)
        data = None 


# Call API to Get Data

def get_data(tipo, nome, pollutant):
    url = "http://127.0.0.1:5000/api/DV_7comune" if tipo == "Comune" else "http://127.0.0.1:5000/api/DV_7provincia"
    payload = {
        "var_comune": nome if tipo == "Comune" else None,
        "var_provincia": nome if tipo == "Provincia" else None,
        "var_pollutant": pollutant
    }
    payload = {k: v for k, v in payload.items() if v is not None}
    response = requests.post(url, json=payload)
    data = read_response(response)
    return pd.DataFrame(data) if data else pd.DataFrame()


# Dropdowns e Output UI (creazione degli elementi interattivi)

tipo_dropdown = widgets.Dropdown(
    options=["Comune", "Provincia"],
    description="Type:",
    value="Comune"
)

nome_dropdown = widgets.Dropdown(
    options=[],
    description="Name:"
)

pollutant_dropdown = widgets.Dropdown(
    options=get_pollutants_list(),
    description="pollutant:"
)

output = widgets.Output()

NameError: name 'get_pollutants_list' is not defined

In [ ]:
######################_____DV_7____######################

# Funzioni per ottenere le opzioni

def get_pollutants():
    t = requests.get("http://127.0.0.1:5000/api/pollutants")
    return [{'label': p, 'value': p} for p in read_response(t)]

def get_municipalities():
    t = requests.get("http://127.0.0.1:5000/api/municipalities")
    return [{'label': m, 'value': m} for m in read_response(t)]

def get_provinces():
    t = requests.get("http://127.0.0.1:5000/api/provinces")
    return [{'label': p, 'value': p} for p in read_response(t)]

# Avvio app Dash
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H2("DV_7: Average pollutant value by municipality or province"),

    html.Div([
        html.Label("Select type (Comune or Provincia):"),
        dcc.RadioItems(
            id='tipo-radio',
            options=[{'label': 'Comune', 'value': 'Comune'}, {'label': 'Provincia', 'value': 'Provincia'}],
            value='Comune',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        )
    ]),

    html.Div([
        html.Label("Select name:"),
        dcc.Dropdown(id='nome-dropdown')
    ], style={'margin-top': '20px'}),

    html.Div([
        html.Label("Select pollutant:"),
        dcc.Dropdown(id='pollutant-dropdown', options=get_pollutants())
    ], style={'margin-top': '20px'}),

    html.Button("Visualize time series", id='submit-button', n_clicks=0, style={'margin-top': '20px'}),

    dcc.Loading(
        id="loading",
        children=dcc.Graph(id='time-series-graph'),
        type="dot"
    )
])

# Callback per aggiornare i nomi
@app.callback(
    Output('nome-dropdown', 'options'),
    Input('tipo-radio', 'value')
)
def update_names(tipo):
    return get_municipalities() if tipo == 'Comune' else get_provinces()

# Callback per ottenere e visualizzare i dati
@app.callback(
    Output('time-series-graph', 'figure'),
    Input('submit-button', 'n_clicks'),
    State('tipo-radio', 'value'),
    State('nome-dropdown', 'value'),
    State('pollutant-dropdown', 'value')
)
def update_graph(n_clicks, tipo, nome, pollutant):
    if n_clicks == 0 or not nome or not pollutant:
        return px.line(title="Please select all inputs and press the button.")

    url = "http://127.0.0.1:5000/api/DV_7comune" if tipo == "Comune" else "http://127.0.0.1:5000/api/DV_7provincia"
    payload = {
        "var_comune": nome if tipo == "Comune" else None,
        "var_provincia": nome if tipo == "Provincia" else None,
        "var_pollutant": pollutant
    }
    payload = {k: v for k, v in payload.items() if v is not None}
    response = requests.post(url, json=payload)
    data = read_response(response)

    if not data:
        return px.line(title=f"No data found for {tipo} '{nome}' and pollutant '{pollutant}'.")

    df = pd.DataFrame(data)
    df['data'] = pd.to_datetime(df['data'])
    df = df.sort_values('data')

    fig = px.line(df, x='data', y='valore', markers=True,
                  title=f"Time series of {pollutant} in {tipo.lower()} {nome}",
                  labels={'valore': 'Average value', 'data': 'Date'})
    return fig

# Run server
if __name__ == '__main__':
    app.run(debug=True)



In [11]:
import dash
from dash import dcc, html, Output, Input, State, ctx, dash_table
import requests
import pandas as pd
import plotly.express as px

# -------- Funzione per leggere risposta --------
def read_response(response):
    try:
        return response.json()
    except Exception as e:
        print("Errore nel parsing della risposta:", e)
        print("Contenuto:", response.text)
        return None

# -------- API Calls --------
def get_pollutants():
    r = requests.get("http://127.0.0.1:5000/api/pollutants")
    return read_response(r) or []

def get_provinces():
    r = requests.get("http://127.0.0.1:5000/api/provinces")
    return read_response(r) or []

def get_municipalities():
    r = requests.get("http://127.0.0.1:5000/api/municipalities")
    return read_response(r) or []

def get_dv7_data(tipo, nome, pollutant):
    url = "http://127.0.0.1:5000/api/DV_7comune" if tipo == "Comune" else "http://127.0.0.1:5000/api/DV_7provincia"
    payload = {
        "var_comune": nome if tipo == "Comune" else None,
        "var_provincia": nome if tipo == "Provincia" else None,
        "var_pollutant": pollutant
    }
    payload = {k: v for k, v in payload.items() if v is not None}
    r = requests.post(url, json=payload)
    return pd.DataFrame(read_response(r) or [])

# -------- DASH App --------
app = dash.Dash(__name__)
server = app.server

pollutant_options = [{"label": p, "value": p} for p in get_pollutants()]

app.layout = html.Div([
    html.H2("DV-7: Media giornaliera per Comune o Provincia"),

    html.Div([
        html.Label("Tipo:"),
        dcc.Dropdown(
            id="tipo-dropdown",
            options=[{"label": "Comune", "value": "Comune"}, {"label": "Provincia", "value": "Provincia"}],
            value="Comune"
        ),
    ], style={'width': '25%', 'display': 'inline-block', 'padding': '0 10px'}),

    html.Div([
        html.Label("Nome:"),
        dcc.Dropdown(id="nome-dropdown"),
    ], style={'width': '35%', 'display': 'inline-block', 'padding': '0 10px'}),

    html.Div([
        html.Label("Inquinante:"),
        dcc.Dropdown(id="pollutant-dropdown", options=pollutant_options),
    ], style={'width': '30%', 'display': 'inline-block', 'padding': '0 10px'}),

    html.Button("Mostra dati", id="show-button", n_clicks=0, style={'margin': '20px'}),

    html.Div(id="alert-message", style={'color': 'red', 'padding': '10px'}),

    html.Div(id="dv7-table"),
    dcc.Graph(id="dv7-graph")
])

# -------- CALLBACK: aggiorna dropdown nomi --------
@app.callback(
    Output("nome-dropdown", "options"),
    Input("tipo-dropdown", "value")
)
def update_nomi(tipo):
    nomi = get_municipalities() if tipo == "Comune" else get_provinces()
    return [{"label": n, "value": n} for n in nomi]

# -------- CALLBACK: mostra dati --------
@app.callback(
    Output("dv7-table", "children"),
    Output("dv7-graph", "figure"),
    Output("alert-message", "children"),
    Input("show-button", "n_clicks"),
    State("tipo-dropdown", "value"),
    State("nome-dropdown", "value"),
    State("pollutant-dropdown", "value")
)
def show_data(n_clicks, tipo, nome, pollutant):
    if not nome:
        return None, {}, "Seleziona un nome valido."
    if not pollutant:
        return None, {}, "Seleziona un inquinante."

    df = get_dv7_data(tipo, nome, pollutant)

    if df.empty:
        return None, {}, f"Nessun dato disponibile per {tipo} '{nome}' e inquinante '{pollutant}'."

    df['data'] = pd.to_datetime(df['data'])
    df = df.sort_values('data')

    table = dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict("records"),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'}
    )

    fig = px.line(df, x='data', y='valore', title=f"Serie temporale di {pollutant} - {tipo}: {nome}")
    fig.update_layout(xaxis_title="Data", yaxis_title="Valore medio")

    return table, fig, ""

# -------- RUN --------
if __name__ == '__main__':
    app.run(debug=True)
